In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
import numpy as np
import cv2
import keras
import matplotlib.pyplot as plt
from keras import applications, metrics, layers, models, regularizers, optimizers
from keras.applications.inception_v3 import InceptionV3
from keras.applications.nasnet import NASNetLarge
from keras.models import load_model
# from skimage.transform import resize
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
nas = NASNetLarge(input_shape=None, include_top=True, weights='imagenet', input_tensor=None, pooling=None, classes=1000)
# nas.summary()
nas.layers.pop()
nas.outputs = [nas.layers[-1].output]
nas.layers[-1].outbound_nodes = []

In [0]:
nas.summary()

In [0]:
inceptionv3 = InceptionV3(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
# print(len(inceptionv3.layers))
inceptionv3.layers.pop()
inceptionv3.outputs = [inceptionv3.layers[-1].output]
inceptionv3.layers[-1].outbound_nodes = []
inceptionv3.summary()

In [0]:
import pickle
import time

In [0]:
width, height, frame_per_segment = 299, 299, 25
path_of_fps15videos = 'gdrive/My Drive/hackathon/Video-cut/train_videos'
n_class = os.listdir(path_of_fps15videos)
class_mapping = {'abnormal': 0,'normal': 1}

trasfer_values = []
labels = []
for single_class in n_class:
  print(single_class)
  single_data = []
  class_path = os.path.join(path_of_fps15videos, single_class)
  videos = os.listdir(class_path)
  for video in videos:
    print(video)
    cap = cv2.VideoCapture(os.path.join(class_path, video))
    ret = True
    count = 0
#     start = int(round(time.time() * 1000))
    trains_image = []
    while ret:
        ret, frame = cap.read()
        if not ret:
          continue
        try:
          image = cv2.resize(frame, (width, height), interpolation = cv2.INTER_AREA)
          trains_image.append(image)
          count += 1
          if count==frame_per_segment:
            count = 0
            transfer_temp = inceptionv3.predict(np.array(trains_image))
#             print(np.array(transfer_temp).shape)
            for transfer in transfer_temp:
              trasfer_values.append(transfer)
              labels.append(class_mapping[single_class])
            del trains_image[:]
        except Exception as e:
            print(str(e))
with open(b"gdrive/My Drive/DA-20182/data/forest_v2.pkl","wb") as f:
    pickle.dump({'data': trasfer_values, 'label': labels},f)
    f.close()

In [0]:
# with open("gdrive/My Drive/DA-20182/data/origin_data/forest_v2.pkl","rb") as f:
#   unpickler = pickle.Unpickler(f)
#   pkl = unpickler.load()
#   data = pkl['data']
#   label = pkl['label']
#   print(len(data), len(label))
#   print(len(data[10000]))

In [0]:
# !sudo pip install git+https://github.com/okankop/vidaug

In [0]:
# from vidaug import augmentors as va

In [0]:
# va.PiecewiseAffineTransform	

In [0]:
width, height, batch_size = 331, 331, 25
video_path = 'gdrive/My Drive/hackathon/Video-cut/train_videos'
n_class = os.listdir(video_path)
class_mapping = {'sam_so': 0,'ko_sam_so': 1}

trasfer_values = []
labels = []
for single_class in n_class:
  print(single_class)
  single_data = []
  class_path = os.path.join(video_path, single_class)
  videos = os.listdir(class_path)
  for video in videos:
    print(video)
    cap = cv2.VideoCapture(os.path.join(class_path, video))
    ret = True
    count = 0
#     start = int(round(time.time() * 1000))
    trains_image = []
    while ret:
      ret, frame = cap.read()
      if not ret:
        continue
      try:
        image = cv2.resize(frame, (width, height), interpolation = cv2.INTER_AREA)
        trains_image.append(image)
        count += 1
        if count==batch_size:
          count = 0
          transfer_temp = nas.predict(np.array(trains_image))
#             print(np.array(transfer_temp).shape)
          for transfer in transfer_temp:
            trasfer_values.append(transfer)
            labels.append(class_mapping[single_class])
          del trains_image[:]
      except Exception as e:
        print(str(e))
with open(b"gdrive/My Drive/hackathon/transfer_value/sex_nas_ver2.pkl","wb") as f:
    pickle.dump({'data': trasfer_values, 'label': labels},f)
    f.close()

NameError: ignored

In [0]:
with open("gdrive/My Drive/hackathon/transfer_value/sex_nas.pkl","rb") as f:
  unpickler = pickle.Unpickler(f)
  pkl = unpickler.load()
  data = pkl['data']
  label = pkl['label']
  print(len(data), len(label))
  print(len(data[10000]))